In [23]:
import numpy as np
import pywt
import matplotlib.pyplot as plt
import random
from scipy import stats
import random
import pywt.data
from PIL import Image
import pandas as pd
import seaborn as sns
import os
import pickle

In [24]:
# assign directory
BRANDON = '/Users/brandonmarks/Desktop/Research Materials/hierarchical-bayesian-model-validation/'
YASH = '/Users/yashd/Desktop/hierarchical-bayesian-model-validation/'

ROOT_DIR = YASH
data_dir = ROOT_DIR + 'raw-data/pastis/full-pastis-grey'
 
# iterate over files in that directory
file_list = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(data_dir)

In [25]:
image = Image.open(file_list[0]).convert('L')
first_image = pywt.wavedec2(image, 'db1')
layer_len = len(first_image)



In [26]:
#for k in range(len(file_list)):
layer_arr = [0] * len(file_list) * (layer_len - 1) * 3 
orientation = [0] * len(file_list) * (layer_len - 1) * 3 
data_arr = [0] * len(file_list) * (layer_len - 1) * 3 
c = 0
for k in range(len(file_list)):
    image = Image.open(file_list[k]).convert('L')
    image = np.array(image)
   
    transformed = pywt.wavedec2(image, 'db1')
    direction_names = ['H', 'V', 'D']

    for i in range(1, layer_len): 
        for j in range(len(transformed[i])):
            
            arr = np.array(transformed[i][j])
            layer_arr[c] = i+1
            orientation[c] =  direction_names[j]
            data_arr[c] = arr.flatten()
            c += 1

In [27]:
df = pd.DataFrame()

df["layer"] = layer_arr
df["orientation"] = orientation
df["data"] = data_arr
new_df = pd.DataFrame(columns=["channel","layer", "orientation", "data"])
for lo, sf in df.groupby(["layer", "orientation"])[["data"]]:#.agg(lambda sf: np.concatenate(sf["Data"].tonumpy()))
    new_df.loc[len(new_df.index)] = ["Red", lo[0], lo[1],  np.hstack(sf['data'])]
new_df

,channel,layer,orientation,data
0,Red,2,D,"[-425.6171874999998, -816.1484375000011, -655...."
1,Red,2,H,"[-54.179687499999915, -304.5703125000002, 337...."
2,Red,2,V,"[-287.7421875, 625.9453124999991, -186.1015625..."
3,Red,3,D,"[255.7343750000003, -512.4375, -597.2187500000..."
4,Red,3,H,"[-169.70312500000065, -201.28125000000028, 675..."
5,Red,3,V,"[209.29687500000045, -239.93750000000045, -106..."
6,Red,4,D,"[-107.12500000000026, -122.03125000000011, 50...."
7,Red,4,H,"[-129.25000000000028, -85.28125000000017, -140..."
8,Red,4,V,"[159.6875, 191.9062500000001, -140.06250000000..."
9,Red,5,D,"[-213.56250000000017, 24.375000000000018, -7.9..."


In [30]:
def convert_to_wavelet_basis(folder_dir, color, basis="db1"):
    file_list = [os.path.join(folder_dir, filename) for filename in os.listdir(folder_dir)]
    #Setup df Dict
    image = Image.open(file_list[0]).convert('L')
    first_image = pywt.wavedec2(image, basis)
    layer_len = len(first_image)
    print(str(layer_len) + " layers being used")
    
    color_dict = {"Red":0, "Green":1, "Blue":2, "Gray":3, "Infrared": 4}
    c = color_dict[color]

    #Fill DF DICT
    layer_arr = [0] * len(file_list) * (layer_len - 1) * 3 
    orientation = [0] * len(file_list) * (layer_len - 1) * 3 
    data_arr = [0] * len(file_list) * (layer_len - 1) * 3 
    cnt = 0
    for k in range(len(file_list)):
        if c >= 3:
            image = np.array(Image.open(file_list[k]).convert('L'))
            std= np.std(image)
            mean = np.mean(image)
            image = (image- mean)/std
        else:
            image = np.array(Image.open(file_list[k]))[:,:,c]
    
        transformed = pywt.wavedec2(image, 'db1')
        direction_names = ['H', 'V', 'D']

        for i in range(1, layer_len): 
            for j in range(len(transformed[i])):
                
                arr = np.array(transformed[i][j])
                layer_arr[cnt] = i+1
                orientation[cnt] =  direction_names[j]
                data_arr[cnt] = arr.flatten()
                cnt += 1

    df = pd.DataFrame()

    df["layer"] = layer_arr
    df["orientation"] = orientation
    df["data"] = data_arr
    new_df = pd.DataFrame(columns=["channel","layer", "orientation", "data"])
    for lo, sf in df.groupby(["layer", "orientation"])[["data"]]:#.agg(lambda sf: np.concatenate(sf["Data"].tonumpy()))
        new_df.loc[len(new_df.index)] = [color, lo[0], lo[1],  np.hstack(sf['data'])]
    
    return new_df

In [31]:
gray_converted = convert_to_wavelet_basis(data_dir, "Gray")
gray_converted

8 layers being used


,channel,layer,orientation,data
0,Gray,2,D,"[-12.878235063866539, -20.30660991719943, -25...."
1,Gray,2,H,"[-1.6393575536980212, -7.578021649152555, 13.1..."
2,Gray,2,V,"[-8.706442402343445, 15.574161153052504, -7.22..."
3,Gray,3,D,"[7.737956764636047, -15.505225762387965, -18.0..."
4,Gray,3,H,"[-5.13484135276545, -6.090325596752094, 20.429..."
5,Gray,3,V,"[6.332860686888242, -7.259978253665985, -32.13..."
6,Gray,4,D,"[-3.2413656490709797, -3.692395816692585, 1.51..."
7,Gray,4,H,"[-3.9108192312011583, -2.5804220701034737, -4...."
8,Gray,4,V,"[4.831790684583638, 5.806658824663036, -4.2379..."
9,Gray,5,D,"[-6.461929068188763, 0.7375336074315529, -0.24..."


In [32]:
os.getcwd()

'c:\\Users\\yashd\\Desktop\\hierarchical-bayesian-model-validation\\dataset-preparation\\pastis'

In [35]:
pd.to_pickle(gray_converted, '../../transformed-data/full-pastis-gray-wavelet.pickle')